In [ ]:
# import library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import random
from scipy import stats

In [ ]:
# setting
plt.rcParams['font.family']= 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial']
plt.rcParams['font.size'] = 18
plt.rcParams["figure.dpi"] = 200

In [ ]:
# list of diel oscillating genes (DOGs)
DOGs_list = []
hm_list = ["H3K4me2","H3K4me3","H2A.Z","H2Aub"]
for hm in hm_list:
    file_path = f"../data/list_of_DOGs/{hm}_DOGs.bed"
    DOGs = pd.read_csv(file_path,sep="\t",header=None).iloc[:,3].values
    DOGs_list.append(DOGs)

file_path = "../data/list_of_DOGs/mRNA_DOGs.txt"
DOGs = pd.read_csv(file_path).columns.values
DOGs_list.append(DOGs)

In [ ]:
# Fig. 5c
hm_list = ["H3K4me2","H3K4me3","H2A.Z","H2Aub","mRNA"]
hm_list_with_n = ["H3K4me2 DOGs\n(n=781)","H3K4me3 DOGs\n(n=1,180)","H2A.Z DOGs\n(n=345)","H2Aub DOGs\n(n=29)","mRNA DOGs\n(n=3,297)"]
N = 27443 # Number of protein-coding genes
kinds_of_DOGs = 5

M = np.zeros([kinds_of_DOGs,kinds_of_DOGs]) # Actual Number of overlapping genes
E = np.zeros([kinds_of_DOGs,kinds_of_DOGs]) # Expected number of overlapping genes
F = np.zeros([kinds_of_DOGs,kinds_of_DOGs]) # Fisher's exact test
J = np.zeros([kinds_of_DOGs,kinds_of_DOGs]) # Jaccard index

for i,X1 in enumerate(DOGs_list):
    for j,X2 in enumerate(DOGs_list):
        n1 = len(X1)
        n2 = len(X2)
        m = len(set(X1) & set(X2))
        e = n1*n2/N
        r = m/e
        M[i][j] = m
        E[i][j] = e
        # print(f"{hm_list[i]}-{hm_list[j]} : {n1},{n2},{m},{e},{r}")
        
        # Fisher's exact test
        f = scipy.stats.fisher_exact(np.array([[m, n1-m],[n2-m,N-(n1+n2-m)]]),alternative='greater')
        F[i][j] = f.pvalue * 10
        
        # Jaccard index
        J[i][j] = m/(n1+n2-m)

fig, ax = plt.subplots(figsize=(3.5,3.5))
for i in range(kinds_of_DOGs):
    for j in range(0,i):
        if i==j:
            continue
        # plot
        ax.scatter(j,kinds_of_DOGs-i-1,s=J[i][j]*2000,c="grey")
        # show results of Fisher's exact test
        text = None
        if F[i][j] < 0.05:
            text = f"*{M[i][j]:.0f}\n ({E[i][j]:.0f})"
        else:
            text = f"{M[i][j]:.0f}\n ({E[i][j]:.0f})"
        ax.text(j,kinds_of_DOGs-i-1,text,size=11,fontname="arial",fontweight="bold")


        
# format figure
ax.set_xlim(-0.5,kinds_of_DOGs-1.5)
ax.set_ylim(-0.5,kinds_of_DOGs-1.5)
ax.set_xticks(np.arange(kinds_of_DOGs-1))
ax.set_yticks(np.arange(kinds_of_DOGs-1))
ax.set_xticklabels(hm_list_with_n[:4],rotation=90,fontsize=10,fontname="arial",fontweight="bold")
ax.set_yticklabels(hm_list_with_n[::-1][:4],fontsize=10,fontname="arial",fontweight="bold")
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.grid()
ax.set_axisbelow(True)

In [ ]:
# H3K4me2 DOGs with different peak time
H3K4me2_DOGs_peak_time_list = []
for i in range(4):
    file = f"../data/list_of_DOGs/H3K4me2_DOGs_ZT{6*i}-peak.bed"
    genes = pd.read_csv(file,sep="\t",header=None).iloc[:,3].values
    H3K4me2_DOGs_peak_time_list.append(genes)
    print(f"ZT{6*i:<2} : {len(genes):>3} genes")

In [ ]:
# Fig. 5d
# Time-course ChIP-seq data
file_path = "../data/ChIP-seq/Time-course-ChIP_WT_rep1.rpkm.tsv"
df_chip = pd.read_csv(file_path, sep="\t", index_col=0)

In [ ]:
# Time-course RNA-seq data
file_path = "../data/RNA-seq/Time-course-RNA_WT_rep1.rpkm.tsv"
df_rna = pd.read_csv(file_path, sep="\t", index_col=0)

# mean of three replicates
df_rna_mean = pd.DataFrame({f"WT_mRNA_ZT{6*i}":df_rna.iloc[:,3*i:3*(i+1)].mean(axis=1) for i in range(4)})

In [ ]:
df_all = df_chip.merge(df_rna_mean,left_index=True,right_index=True)

In [ ]:
hm_list = ["H3","H3K4me2","H3K4me3","H2A.Z","H2Aub","mRNA"]
color_list = ["tab:grey","tab:orange","tab:cyan","tab:red","tab:green","tab:purple"]

hm_n = 6
zt_n = 4

fig, ax = plt.subplots(hm_n,zt_n,figsize=(18,18),sharex="col",sharey="row",dpi=300)

for i in range(hm_n):
    hm = hm_list[i]
    c = color_list[i]
    for j in range(zt_n):
        mask = df_all.index.isin(H3K4me2_DOGs_peak_time_list[j])
        X = df_all.iloc[:,4*i:4*(i+1)].loc[mask]
        
        # add ZT0 as ZT24
        X[f"WT_{hm}_ZT24"] = X.iloc[:,0]
        X.columns = np.arange(zt_n+1)
        
        # standardization
        std_X = X.apply(lambda x: (x - x[:4].mean())/x[:4].std(ddof=1),axis=1)

        # plot
        ax[i][j].plot(std_X.T,c=c,alpha=0.1,linewidth=1)
        ax[i][j].plot(std_X.mean(),c="k",alpha=1,linewidth=2,linestyle="--",
                      marker="o")
        
for j in range(4):
    ax[0][j].set_xticks(np.arange(zt_n+1))
    ax[0][j].set_xticklabels(["ZT0","ZT6","ZT12","ZT18","(ZT0)"])

In [ ]:
# Fig. 5e
N = 6

r_list = []
hm_list = ["H3","H3K4me2","H3K4me3","H2A.Z","H2Aub","mRNA"]
hm_n = 6

mask = df_all.index.isin(DOGs_list[0]) # H3K4me2 DOGs (n=781)

for i in range(hm_n):
    if i == 1:
        continue
        
    hm = hm_list[i]
    
    df_tmp1 = df_all.iloc[:,4:8][mask]
    df_tmp2 = df_all.iloc[:,4*i:4*(i+1)][mask]
  
    # Spearman's R 
    R = []
    for j in range(len(df_tmp1)):
        x = df_tmp1.iloc[j,:]
        y = df_tmp2.iloc[j,:]
        R.append(pearsonr(x,y)[0])
    r_list.append(R)

In [ ]:
# plot
color_list = ["tab:grey","tab:cyan","tab:red","tab:green","tab:purple"]
fig, ax = plt.subplots(1,5,figsize=(22,4))

for i in range(5):
    ax[i].hist(r_list[i],bins=20,range=(-1,1),color=color_list[i],ec='black')

In [ ]:
# Fig. 5f
# Time-course ChIP-seq data in WT and atxr3
file_path = "../data/ChIP-seq/Time-course-ChIP_WT_atxr3_rep1.rpkm.tsv"
df_chip = pd.read_csv(file_path, sep="\t", index_col=0)

# Time-course mRNA-seq data in WT and atxr3
file_path = "../data/RNA-seq/Time-course-RNA_WT_atxr3.rpkm.tsv"
df_rna = pd.read_csv(file_path, sep="\t", index_col=0)
# mean of three replicates
gt_list = ["WT","atxr3"]
df_rna_mean = np.sqrt(pd.DataFrame({f"{gt_list[i]}_mRNA_ZT{6*j}":df_rna.iloc[:,12*i+3*j:12*i+3*(j+1)].mean(axis=1) for i in range(2) for j in range(4)}))

In [ ]:
df_all = df_chip.iloc[:,:16].merge(df_rna_mean.iloc[:,:4],left_index=True,right_index=True).merge(
    df_chip.iloc[:,16:],left_index=True,right_index=True).merge(
    df_rna_mean.iloc[:,4:],left_index=True,right_index=True)
df_all.head()

In [ ]:
hm_list = ["H3","H3K4me2","H3K4me3","H2A.Z","mRNA"]
c_list = ["k","tab:orange","tab:cyan","tab:red","tab:purple"]

max_list = [6,10,20,5,5]
min_list = [-6,-30,-40,-10,-5]

mask = df_chip.index.isin(DOGs_list[0]) # H3K4me2 DOGs
N = mask.sum()

# H3K4me2 non-DOGs
random.seed(1)
mask_nonCOGs = df_chip.index.isin(random.sample(list(df_chip.index[~mask]),N))
#print(N,df_chip[mask_nonCOGs].index[:3])


fig, ax = plt.subplots(1,5,figsize=(25,5))
for i in range(5):
    hm = hm_list[i]
    ser_wt = df_all.iloc[:,4*i:4*(i+1)].max(axis=1) - df_all.iloc[:,4*i:4*(i+1)].min(axis=1)
    ser_mt = df_all.iloc[:,4*(i+5):4*(i+6)].max(axis=1) - df_all.iloc[:,4*(i+5):4*(i+6)].min(axis=1)

    # H3K4me2 DOGs
    a = ser_wt[mask]
    b = ser_mt[mask]
    
    # H3K4me2 non-DOGs
    c = ser_wt[mask_nonCOGs]
    d = ser_mt[mask_nonCOGs]

    # Amplitude change (atxr3 - WT)
    e = b-a
    f = d-c

    # plot
    sns.violinplot({"1":f,
                    "2":e},
                   palette = ["tab:grey",c_list[i]],
                   fill=False,
                   cut=0,
                   ax=ax[i],
                   inner_kws=dict(box_width=10,whis_width=2,),
               )

    # format figure
    ax[i].set_ylim(min_list[i],max_list[i])
    ax[i].spines['right'].set_visible(False)
    ax[i].spines['top'].set_visible(False)
    ax[i].hlines(0,-0.5,1.5,color="black",linestyle="--",zorder=0)
    ax[i].set_xlim(-0.5,1.5)
    ax[i].set_xticks([0,1])
    ax[i].set_xticklabels(["H3K4me2\nnon-DOGs","H3K4me2\nDOGs"])
    ax[i].set_title(hm)
    
    # Mann-Whitney U test
    p1 = stats.mannwhitneyu(f, e, alternative="two-sided")[1]
    print(f"{hm:10s} p={p1:.2e}")